In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
# 모듈화, 계층화 재사용
# residual 개수 변경
inputs = keras.Input(shape=(32,32,3))
x = layers.Conv2D(32,3,activation='relu')(inputs)
residual = x
x = layers.Conv2D(64,3,activation='relu',padding='same')(x)
x = layers.MaxPool2D(2,padding='same')(x)
residual = layers.Conv2D(64,1,strides=2)(residual)
x = layers.add([x,residual])

In [12]:
inputs = keras.Input(shape=(32,32,3))
x = layers.Rescaling(1./255.)(inputs)
def residual_block(x, filters, pooling = False):
  residual = x
  x = layers.Conv2D(filters,3,activation='relu',padding='same')(x)
  x = layers.Conv2D(filters,3,activation='relu',padding='same')(x)
  if pooling:
    x = layers.MaxPool2D(2,padding='same')(x)
    residual = layers.Conv2D(filters,1,strides=2)(residual)
  elif filters != residual.shape[-1]:
    residual = layers.Conv2D(filters,1)(residual)
  x = layers.add([x, residual])
  return x


In [13]:
x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=True)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_6[0][0]']                
                                                                                                  
 conv2d_15 (Conv2D)             (None, 32, 32, 32)   896         ['rescaling[0][0]']              
                                                                                                  
 conv2d_16 (Conv2D)             (None, 32, 32, 32)   9248        ['conv2d_15[0][0]']              
                                                                                            

In [ ]:
# 적용 예정(개 고양이 데이터)

In [14]:
# 학습을 원할하게 해 주는 배치 정규화

In [4]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    "/content/drive/MyDrive/train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    "/content/drive/MyDrive/validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    "/content/drive/MyDrive/test",
    image_size=(180, 180),
    batch_size=32)

Found 1000 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [16]:
# Xception 유사한 모델
# 데이터 제너러이터를 이용
data_generator = keras.Sequential(
    [
      keras.layers.RandomFlip('horizontal'),
      keras.layers.RandomRotation(0.1),
      keras.layers.RandomZoom(0.2)
    ]    
)
inputs = keras.layers.Input(shape = (180,180,3))
x = data_generator(inputs)
x = layers.Rescaling(1./255.)(x)
x = layers.Conv2D(32,5,use_bias=False )(x)
for size in [32,64,128,256,512]:
  residual = x
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)

  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)

  x = layers.MaxPool2D(3,strides=2,padding = 'same')(x)

  residual = layers.Conv2D(size,1,strides=2,padding='same', use_bias=False)(residual)
  x = layers.add([x, residual])
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs,outputs)

In [12]:
class Test:
  def __init__(self) -> None:
    self.data = 100    
class Test2:
  def __init__(self) -> None:
    self.data2 = 100      

In [13]:
x = Test()
residual = x
x = Test2()

In [14]:
residual, x

(<__main__.Test at 0x7f39800795e0>, <__main__.Test2 at 0x7f3980079460>)

In [18]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset)

Epoch 1/30
32/32 [==============================] - 144s 4s/step - loss: 0.7033 - accuracy: 0.5770 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/30
32/32 [==============================] - 17s 516ms/step - loss: 0.6703 - accuracy: 0.5770 - val_loss: 0.6920 - val_accuracy: 0.5440
Epoch 3/30
32/32 [==============================] - 16s 492ms/step - loss: 0.6470 - accuracy: 0.6110 - val_loss: 0.6953 - val_accuracy: 0.4980
Epoch 4/30
32/32 [==============================] - 16s 485ms/step - loss: 0.6537 - accuracy: 0.6120 - val_loss: 0.7176 - val_accuracy: 0.5000
Epoch 5/30
32/32 [==============================] - 16s 474ms/step - loss: 0.6442 - accuracy: 0.6180 - val_loss: 0.6921 - val_accuracy: 0.5040
Epoch 6/30
32/32 [==============================] - 16s 483ms/step - loss: 0.6411 - accuracy: 0.6500 - val_loss: 0.7400 - val_accuracy: 0.5000
Epoch 7/30
32/32 [==============================] - 16s 487ms/step - loss: 0.6254 - accuracy: 0.6380 - val_loss: 0.7085 - val_accuracy: 0.5000
E

KeyboardInterrupt: ignored